In [1]:
import sys
import  os
from .request import Request
from .get_stream import GetStream
from .history import  History
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

ImportError: attempted relative import with no known parent package

In [ ]:
def download_chunk(url, chunk_size=4096):
    with Request().get(url, stream=True) as r:
        chunk = r.iter_content(chunk_size=chunk_size)
        return b"".join(chunk)

In [ ]:
def download_file(url, file_path, num_threads=4):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        response = Request().head(url)
        total_size = int(response.headers.get('content-length', 0))

        with open(file_path, "wb") as f:
            futures = []
            for i in range(0, total_size, chunk_size * num_threads):
                futures.append(executor.submit(download_chunk, url, chunk_size))
            for future in futures:
                f.write(future.result())

In [ ]:
def __download( download_data):
    if download_data['stream_url']:
        print (download_data['file_name'])
        History().status = "run"
        fullpath = os.path.join(os.path.curdir, download_data['file_name'])

        with Request().get(download_data['stream_url'], stream=True) as r, open(fullpath, "wb") as f, tqdm(
                unit="B",
                unit_scale=True,
                unit_divisor=1024,
                total=int(r.headers.get('content-length', 0)),
                file=sys.stdout,
                desc=download_data['file_name']
        ) as progress:
            for chunk in r.iter_content(chunk_size=4096):
                if chunk:
                    datasize = f.write(chunk)
                    progress.update(datasize)
        History().run_episode = History().run_episode + 1